# NVD Data Quality

In [7]:
from IPython.display import display
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import glob

## Load and Parse CVE Data

In [8]:
def get_nested_value(entry, keys, default=np.nan):
    try:
        for key in keys:
            entry = entry[key]
        return entry
    except (KeyError, IndexError):
        return default

row_accumulator = []
for filename in glob.glob('../Data/NVD/nvd.jsonl'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for entry in nvd_data:
            new_row = {
                'CVE': get_nested_value(entry, ['cve', 'id']),
                'Published': get_nested_value(entry, ['cve', 'published']),
                'AttackVector': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'attackVector']),
                'AttackComplexity': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'attackComplexity']),
                'PrivilegesRequired': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'privilegesRequired']),
                'UserInteraction': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'userInteraction']),
                'Scope': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'scope']),
                'ConfidentialityImpact': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'confidentialityImpact']),
                'IntegrityImpact': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'integrityImpact']),
                'AvailabilityImpact': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'availabilityImpact']),
                'BaseScore': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'baseScore'], '0.0'),
                'BaseSeverity': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'cvssData', 'baseSeverity']),
                'ExploitabilityScore': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'exploitabilityScore']),
                'ImpactScore': get_nested_value(entry, ['cve', 'metrics', 'cvssMetricV31', 0, 'impactScore']),
                'CWE': get_nested_value(entry, ['cve', 'weaknesses', 0, 'description', 0, 'value']),
                'Description': get_nested_value(entry, ['cve', 'descriptions', 0, 'value'], ''),
                'Assigner': get_nested_value(entry, ['cve', 'sourceIdentifier']),
                'Tag': get_nested_value(entry, ['cve', 'cveTags', 0, 'tags'], np.nan),
                'Status': get_nested_value(entry, ['cve', 'vulnStatus'], '')
            }
            row_accumulator.append(new_row)

nvd = pd.DataFrame(row_accumulator)
nvd = nvd[~nvd.Status.str.contains('Rejected')]
nvd['Published'] = pd.to_datetime(nvd['Published'])
nvd = nvd.sort_values(by=['Published'])
nvd = nvd.reset_index(drop=True)
nvd['BaseScore'] = pd.to_numeric(nvd['BaseScore']);
nvd['BaseScore'] = pd.to_numeric(nvd['BaseScore']);
nvd['BaseScore'] = nvd['BaseScore'].replace(0, np.nan);


In [9]:
# Generate a data quality report
def data_quality_report(df):
    report = pd.DataFrame({
        'Column': nvd.columns,
        'Non-Null Count': nvd.notnull().sum(),
        'Null Count': nvd.isnull().sum(),
        'Unique Count': nvd.apply(lambda x: x.dropna().apply(str).nunique()),
        'Data Type': nvd.dtypes
    })
    
    # Add basic statistics for numeric columns, excluding list-type columns
    numeric_columns = nvd.select_dtypes(include=[np.number]).columns
    numeric_stats = nvd[numeric_columns].describe().transpose()
    numeric_stats = numeric_stats[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
    numeric_stats.columns = ['Mean', 'Std Dev', 'Min', '25%', '50%', '75%', 'Max']
    
    # Merge the numeric stats with the report
    report = report.merge(numeric_stats, left_on='Column', right_index=True, how='left')
    
    return report

# Generate the report
report = data_quality_report(nvd)

# Display the report without the index
display(report.style.hide(axis="index"))

Column,Non-Null Count,Null Count,Unique Count,Data Type,Mean,Std Dev,Min,25%,50%,75%,Max
CVE,273138,0,273138,object,nan,nan,nan,nan,nan,nan,nan
Published,273138,0,244729,datetime64[ns],nan,nan,nan,nan,nan,nan,nan
AttackVector,154579,118559,4,object,nan,nan,nan,nan,nan,nan,nan
AttackComplexity,154579,118559,2,object,nan,nan,nan,nan,nan,nan,nan
PrivilegesRequired,154579,118559,3,object,nan,nan,nan,nan,nan,nan,nan
UserInteraction,154579,118559,2,object,nan,nan,nan,nan,nan,nan,nan
Scope,154579,118559,2,object,nan,nan,nan,nan,nan,nan,nan
ConfidentialityImpact,154579,118559,3,object,nan,nan,nan,nan,nan,nan,nan
IntegrityImpact,154579,118559,3,object,nan,nan,nan,nan,nan,nan,nan
AvailabilityImpact,154579,118559,3,object,nan,nan,nan,nan,nan,nan,nan
